In [1]:
from configparser import ConfigParser
import psycopg2.extras
import pandas as pd
import os
import requests 
import zipfile

In [2]:
#sign into database


In [5]:
with conn.cursor() as cursor:
        print('PostgreSQL database version:')
        cursor.execute('SELECT version()')
        # display the PostgreSQL database server version
        db_version = cursor.fetchone()
        print(db_version)
        cursor.execute ("select id, created_date as date from caltrans_cctv_archive order by date asc limit 100")
        list = cursor.fetchall()
        df = pd.DataFrame(list, columns = ["Id","Date"])

PostgreSQL database version:
('PostgreSQL 16.3 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 7.3.1 20180712 (Red Hat 7.3.1-6), 64-bit',)


In [6]:
for index,row in df.iterrows():
    
    #url to get cctv image data
    test_string = "https://api.traffic.labrador.dev/caltrans_cctv_archive/" + row['Id'] 
    
    #converting  file name to zip
    file_name = row['Id'] + ".zip" 
    
    #save path to folder
    save_path = os.getcwd() + "\\unlabeled_data" + "/" + file_name

    #download function
    def download_url(url, save_path, chunk_size=128):
        r = requests.get(url, stream=True)
        with open(save_path, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=chunk_size):
                fd.write(chunk)
    
    download_url(test_string,save_path)

    # unzip the file
    with zipfile.ZipFile(save_path, 'r') as zip_ref:
        zip_ref.extractall(os.path.join(os.getcwd(), "unlabeled_data", row['Id']))

    # delete the zip file after extraction
    os.remove(save_path)
    